In [1]:
from werkzeug.wrappers import Request, Response
from flask import Flask,redirect, url_for, request
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import os
import requests
from flask import Flask, render_template, request


app = Flask(__name__, template_folder='template')

@app.route("/", methods=['GET', 'POST'])

def hello():
    if request.method == "POST":
        try:
            text = request.form['fname']
            
            #r = requests.get(url)
            return redirect(url_for('summary',article_text = text))
        except:
            errors.append("Please add the text")
            return render_template('upload1.html', errors=errors)
        
    return render_template('upload1.html')


@app.route("/<article_text>")
def summary(article_text):
    
    article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)  
    article_text = re.sub(r'\s+', ' ', article_text) 
    formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )  
    formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)
    sentence_list = nltk.sent_tokenize(article_text)
    clean_sentences = pd.Series(sentence_list).str.replace("[^a-zA-Z]", " ")
    clean_sentences = [s.lower() for s in clean_sentences]
    stop_words = stopwords.words('english')
    def remove_stopwords(sen):
        sen_new = " ".join([i for i in sen if i not in stop_words])
        return sen_new


    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

    word_embeddings = {}
    f = open('C:/users/Videet/1/glove.6B.100d.txt', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()

    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)
    sim_mat = np.zeros([len(sentence_list), len(sentence_list)])
    for i in range(len(sentence_list)):
        for j in range(len(sentence_list)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentence_list)), reverse=True)
    for i in range(1):
        return(ranked_sentences[i][1])
    
if __name__ == '__main__':
    
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Jul/2019 18:06:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jul/2019 18:06:51] "GET /1.jpg HTTP/1.1" 200 -
127.0.0.1 - - [03/Jul/2019 18:07:13] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [03/Jul/2019 18:07:30] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [03/Jul/2019 18:07:52] "GET /HC%20gets%20medical%20report%20of%20no%20birthmark%20on%20Dhanush%27s%20body%2CThe%20Madras%20High%20Court%20has%20examined%20the%20medical%20report%20where%20no%20birthmark%20or%20scar%20has%20been%20found%20on%20actor%20Dhanush%27s%20body%2C%20as%20mentioned%20by%20the%20elderly%20couple%20claiming%20to%20his%20parents.%20However%2C%20the%20report%20stated%20that%20a%20small%20superficial%20mole%20and%20another%20large%20mole%20had%20been%20removed.%20The%20hearing%20in%20the%20case%20has%20now%20been%20deferred%20to%20March%2027.%22 HTTP/1.1" 200 -
127.0.0.1 - - [03/Jul/2019 18:08:15] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 -